# Visualizing scattering in Union components

Similar to absorption loggers we have regular loggers! These record scattered intensity, and have a few more options, hence we started with the simpler absorption loggers. This presentation is about loggers.

## Union in the instrument
Since everything happens in the Union master component, we can't insert monitors between the geometries.

![alt](figures/power_point_figures/Slide04.png)

## Complexity from scattering and geometry
We want to understand how the complex results arise from the relatively simple geometry and rules of scattering.

![alt](figures/cryostat_geometry.png)

## Remember our system
Here is an overview of the system we built in the last session

![alt](figures/power_point_figures/Slide05.png)

In [ ]:
import mcstasscript as ms
instrument = ms.McStas_instr("Union_demo", input_path="run_folder")
instrument.settings(output_path="data_folder/union_demo")

instrument.add_component("init", "Union_init")

Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.set_parameters(sigma=4*0.0082, unit_cell_volume=66.4)

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = '"Al.laz"'

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = 100*4*0.231/66.4

process = instrument.add_component("diamond_incoherent", "Incoherent_process")
process.set_parameters(sigma=8*0.001, unit_cell_volume=45.39)

instrument.add_parameter("diamond_phi", value=0, comment="Rotation of diamond lattice around z [deg]")
crystal_process = instrument.add_component("diamond_crystal", "Single_crystal_process")
crystal_process.set_parameters(mosaic=20, reflections='"C_diamond.lau"',
                               ax=3.567, by=3.567, cz=3.567,)
crystal_process.set_ROTATED([0, 0, "diamond_phi"])

diamond = instrument.add_component("diamond", "Union_make_material")
diamond.process_string = '"diamond_incoherent,diamond_crystal"'
diamond.my_absorption = 100*8*0.0035/45.39

sample_position = instrument.add_component("sample_position", "Arm")
sample_position.set_AT([0,0,1])

instrument.add_parameter("A3", value=0, comment="Rotation of sphere around y [deg]")
sphere = instrument.add_component("sphere", "Union_sphere")
sphere.radius = 0.01
sphere.material_string = '"diamond"'
sphere.priority = 10
sphere.set_AT(0, RELATIVE=sample_position)
sphere.set_ROTATED([0, "A3", 0], RELATIVE=sample_position)

shell = instrument.add_component("shell", "Union_cylinder")
shell.set_parameters(radius=0.05, yheight=0.12,
                     material_string = '"Al"', priority=3,
                     p_interact=0.2)
shell.set_AT(0, RELATIVE=sample_position)

shell_vacuum = instrument.add_component("shell_vacuum", "Union_cylinder")
shell_vacuum.set_parameters(radius=shell.radius - 0.01, yheight=shell.yheight - 0.01,
                            material_string = '"Vacuum"', priority=5)
shell_vacuum.set_AT([0, 0, 0], RELATIVE=sample_position)

source = instrument.add_component("source", "Source_div")
source.set_parameters(focus_aw=0.3, focus_ah=0.3,
                      xwidth=0.005, yheight=0.005,
                      flux=1E12)
source.set_AT(-1, RELATIVE=sample_position)

source.lambda0 = instrument.add_parameter("wavelength", value=5.0,
                                          comment="Wavelength in [Ang]")
source.dlambda = instrument.add_parameter("wavelength_half_width", value=4.5,
                                          comment="Wavelength half width in [Ang]")

master = instrument.add_component("master", "Union_master")
stop = instrument.add_component("stop", "Union_stop")

## Logger components
Logger components record something whenever a scattering event occurs in the simulated Union system. Like absorption loggers there are a variety of loggers available and it requires little effort to add a new logger.

| Logger component           | Dimensions                            |
| :---                       |    :----                              |
| Union_logger_1D            | time, magnitude of scattering vector  |
| Union_logger_2DQ           | 2D scattering vector                  |
| Union_logger_2D_kf         | 2D final wavevector                   |
| Union_logger_2D_kf_time    | 2D final wavevector and time          |
| Union_logger_2D_space      | 2D space                              |
| Union_logger_2D_space_time | 2D space and time                     |
| Union_logger_3D_space      | 3D space                              |

### Common inputs on all loggers
Some inputs are common for all Union logger components.

| Parameter                  | Explanation                                                  |
| :---                       |    :----                                                     |
| target_geometry            | Only record for given list of geometries                     |
| target_process             | Further select processes (only if geometry selected)              |
| order_total                | Record only the n'th scattering of rays                      |
| order_volume               | Record only the n'th scattering in the same geometry         |
| order_volume_process       | Record only the n'th scattering in same geometry and process |

### Model of the BIFROST collimation + filter using Union_logger_2D_space
![alt](figures/BIFROST_filter.png)

### Adding a few logger components to our example
Back to our example with the diamond sphere within the aluminium shell.

In [ ]:
logger_time = instrument.add_component("logger_time_all", "Union_logger_1D", before=master)
logger_time.set_parameters(min_value=0, max_value=0.003, n1=600, filename='"scattering_time.dat"')

logger_time = instrument.add_component("logger_time_shell", "Union_logger_1D", before=master)
logger_time.set_parameters(min_value=0, max_value=0.003, n1=600, filename='"scattering_time_shell.dat"',
                           target_geometry='"shell"')

logger_zx = instrument.add_component("logger_space_zx", "Union_logger_2D_space",
                                     before=master, RELATIVE=sample_position)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx.dat"')

logger_zx = instrument.add_component("logger_space_zx_1", "Union_logger_2D_space",
                                     before=master, RELATIVE=sample_position)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx_1.dat"', order_total=1)

logger_zx = instrument.add_component("logger_space_zx_2", "Union_logger_2D_space",
                                     before=master, RELATIVE=sample_position)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zx_2.dat"', order_total=2)

### Adding a few more logger components to our example

In [ ]:
logger_zy = instrument.add_component("logger_space_zy", "Union_logger_2D_space",
                                     before=master, RELATIVE=sample_position)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_zy.dat"')

logger_xy = instrument.add_component("logger_space_xy", "Union_logger_2D_space",
                                     before=master, RELATIVE=sample_position)
logger_xy.set_parameters(D_direction_1='"x"', D1_min=-0.06, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         filename='"logger_xy.dat"') 

logger_2DQ = instrument.add_component("logger_2DQ", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min = -8.0, Q1_max = 8.0, n1 = 500,
                         Q_direction_2 = '"x"', Q2_min = -8.0, Q2_max = 8.0, n2 = 500,
                         filename = '"logger_2DQ.dat"')

### And the normal monitors

In [ ]:
instrument.add_declare_var("double", "t_min")
instrument.add_declare_var("double", "t_max")

instrument.append_initialize("t_min = (wavelength - wavelength_half_width)*(1.0 - 0.18 + 0.5)/(K2V*2*PI);")
instrument.append_initialize("t_max = (wavelength + wavelength_half_width)*(1.0 + 0.28 + 0.5)/(K2V*2*PI);")

instrument.add_declare_var("char", "options", array=256)
instrument.append_initialize('sprintf(options,"banana, theta limits=[-180,180] bins=180, t limits=[%g %g] bins=300",t_min,t_max);')

banana_detector = instrument.add_component("banana_tof_detector", "Monitor_nD")
banana_detector.set_RELATIVE(sample_position)
banana_detector.xwidth = 1
banana_detector.yheight = 0.2
banana_detector.restore_neutron = 1
options = "options"
banana_detector.options = options
banana_detector.filename = '"tof_b.dat"'

banana_detector = instrument.add_component("banana_detector", "Monitor_nD")
banana_detector.set_RELATIVE(sample_position)
banana_detector.set_parameters(xwidth = 1.0, yheight = 0.2, filename='"banana.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=180"')

L_monitor = instrument.add_component("L_monitor", "L_monitor")
L_monitor.set_AT(0.3, RELATIVE=sample_position)
L_monitor.set_parameters(xwidth=0.3, yheight=0.3,
                         Lmin="wavelength - wavelength_half_width",
                         Lmax="wavelength + wavelength_half_width",
                         nL=300, filename='"l_mon.dat"', restore_neutron=1)

## Running the simulation

In [ ]:
import mcstasscript.jb_interface as ms_widget
%matplotlib widget
ms_widget.show(instrument)

## Using target_process
The 2DQ logger had a lot of information, lets split it into its different parts. This could also be further done with scattering order. 

In [ ]:
logger_2DQ = instrument.copy_component("logger_2DQ_shell", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Al.dat"', target_geometry='"shell"')

logger_2DQ = instrument.copy_component("logger_2DQ_Al_inc", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Al_inc.dat"', target_geometry='"shell"',
                          target_process='"Al_incoherent"')

logger_2DQ = instrument.copy_component("logger_2DQ_Al_pow", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Al_pw.dat"', target_geometry='"shell"',
                          target_process='"Al_powder"')

logger_2DQ = instrument.copy_component("logger_2DQ_Diamond", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Diamond.dat"', target_geometry='"sphere"')

logger_2DQ = instrument.copy_component("logger_2DQ_Diamond_inc", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Diamond_inc.dat"', target_geometry='"sphere"',
                          target_process='"diamond_incoherent"')

logger_2DQ = instrument.copy_component("logger_2DQ_Diamond_single", "logger_2DQ", before=master)
logger_2DQ.set_parameters(filename = '"logger_2DQ_Diamond_single.dat"', target_geometry='"sphere"',
                          target_process='"diamond_crystal"')

### Run the instrument again

In [ ]:
ms_widget.show(instrument)

### Making an animation from time logger 

In [ ]:
instrument.add_declare_var("double", "t_min_system")
instrument.add_declare_var("double", "t_max_system")
instrument.append_initialize("t_min_system = (wavelength - wavelength_half_width)*(1.0 - 0.071)/(K2V*2*PI);")
instrument.append_initialize("t_max_system = (wavelength + wavelength_half_width)*(1.0 + 0.28)/(K2V*2*PI);")

logger_zy = instrument.add_component("logger_2D_space_time", "Union_logger_2D_space_time",
                                     before=master, RELATIVE=sphere)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=300,
                         D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                         time_bins=70, time_min="t_min_system", time_max="t_max_system",
                         filename='"logger_zy_time.dat"')

instrument.set_parameters(wavelength=2.38, wavelength_half_width=0.03)
instrument.settings(ncount=5E7, mpi=6, force_compile=True)
data = instrument.backengine()

### Plot the result
McStasScript has a convenient function for making animations!

In [ ]:
ani_data = ms.name_search("logger_2D_space_time", data)
    
ms.make_animation(ani_data, filename="animation_demo", fps=3,
                  log=True, colormap="hot", orders_of_mag=6, figsize=(8, 5))

### Cryostat simulation
More detailed simulation with higher ncount performed on cluster
<table><tr>
<td> <img src="figures/cryostat_geometry.png" alt="Cryostat sketch and data" style="width: 600px;"/> </td>
<td> <img src="figures/output.gif" alt="Cryostat animation" style="width: 900px;"/> </td>
</tr></table>

### Pressure cell with 2 diamonds and powder sample
Tof pulse around diamond bragg peak
![alt](figures/anvil_bragg.gif)

### Simulated detector signal when rotating one of the diamonds
![alt](figures/full_rotation_movie_2.gif)

# Union overview so far
Now we have learnt about 7 classes of Union components!

| Component class         | Links to                       | Purpose                                    |
| :---                    |    :----                       |    :----                                   |
| Process components      |                                | Describe scattering process                |
| Union_make_material     | List of process components     | Collect process and absorption to material |
| Geometry components     | Material definition            | Place a geometry with selected material    |
| Logger components       | Optionally geometry / process  | Record scattering in Union system          |
| Abs_logger components   | Optionally geometry / process  | Record absorption in Union system          |
| Union_master            | Geometries (automatic linking) | Simulation engine                          |
| Union_init / Union_stop |                                | Needed in McStas 3.X at start / end        |



## Summary
Loggers help us understand what goes on during the simulation of a Union system
- They record information at points of scattering
- Can select to only record for given list of geometries
- Can select to only record for given list of processes within those geometries
- Can select certain order as well

## Quiz and exercise
It is time for exercise 5 where the cryostat tool in McStasScript is used to provide an interesting sample environment for a single crystal sample.